# Audit qualité données Aix Faubourg
Auditer la précision des données saisies sur Aix Faubourg.

Dans ce notebook, on :

- Calcule le temps passé dans chaque zone
- Le nombre d'objets relevés dans chaque zone
- Le temps par objet
- La précision dans chaque zone
- La précision obtenue par classe d'objet

Nous verrons que la première zone est celle qui possède les meilleurs scores (zone dégagée souvent en RTK fix) avec des précisions de 1cm et que les arbres obtiennent des scores médiocres en précision, ce qui rend leur inventaire délicat via le rover RTK.

On lit les 2 gpkgs

In [16]:
import geopandas as gpd
import fiona
import pandas as pd

gpkg1 = '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
gpkg2 = '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'

In [17]:
layers = fiona.listlayers(gpkg1)
layers

['arbres',
 'avaloirs',
 'bancs',
 'barri_res_points',
 'blocs_de_pierres_ou_b_ton',
 'borne_amovible',
 'bornes_incendie',
 'camera',
 'fontaine',
 'jardini_res',
 'Panneaux',
 'passage_pi_tons',
 'Potelet',
 'trottoir_ligne',
 'trottoir_poly',
 '_clairage',
 'Photos',
 'Tracks',
 'TrackPoints',
 'android_metadata']

## Test sur les arbres

In [18]:
data = gpd.read_file(gpkg1, layer = 'arbres').set_crs('epsg:4326')
h_accuracies = data.h_accuracy.to_list()
data_types = ['arbres'] * len(h_accuracies) 
print(h_accuracies)
print(data_types)

[0.067, 0.139, 0.381, 0.12]
['arbres', 'arbres', 'arbres', 'arbres']


C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(


### Intersection avec les arbres

In [19]:
zones = gpd.read_file('../terrain_data/raw/Aix Faubourg/zones.gpkg').set_crs('epsg:4326')
zones

,geometry
0,"POLYGON ((5.44347 43.53157, 5.44342 43.5313, 5..."
1,"POLYGON ((5.44371 43.52974, 5.44411 43.52963, ..."
2,"POLYGON ((5.43996 43.53083, 5.4404 43.53093, 5..."
3,"POLYGON ((5.44139 43.53011, 5.44133 43.52996, ..."


In [20]:
data.intersects(zones).to_list()

[False, False, False, False]

### Test export

In [21]:
data.drop(columns=['FID']).to_file('test_arbres.gpkg', driver = 'GPKG')
zones.to_file('zones.gpkg', driver = 'GPKG')

In [22]:
data.geometry.geom_type

0    Point
1    Point
2    Point
3    Point
dtype: object

In [23]:
zones.geometry.geom_type

0    Polygon
1    Polygon
2    Polygon
3    Polygon
dtype: object

In [24]:
zones.iloc[0]

geometry    POLYGON ((5.44347 43.53157, 5.44342 43.5313, 5...
Name: 0, dtype: geometry

In [25]:
data.geometry

0    POINT Z (6.36354 43.49211 249.24827)
1    POINT Z (6.36358 43.49211 247.62757)
2    POINT Z (6.36351 43.49211 247.43425)
3      POINT Z (5.44357 43.53137 249.705)
Name: geometry, dtype: geometry

In [26]:
data.geometry.intersects(zones.iloc[0].geometry)

0    False
1    False
2    False
3     True
dtype: bool

In [27]:
res = data.iloc[0].geometry.intersects(zones.geometry)
res[res].index.values.flatten()

array([], dtype=int64)

In [28]:
res = data.iloc[3].geometry.intersects(zones.geometry)
values = res[res].index.values
values

array([0])

Première zone intersectée par l'objet.

In [29]:
len(values)

1

## data 1

In [30]:
data_types = list()
id_zones = list()
h_accuracies = list()
longitudes = list()
latitudes = list()
timestamps = list()

for layer in layers:
    print(layer)
    try:
        data = gpd.read_file(gpkg1, layer = layer)
        cols = data.columns
        if 'h_accuracy' in cols:
            n = data.shape[0]
            for i in range(n):
                row = data.iloc[i]
                res = row.geometry.intersects(zones.geometry)
                id_zone = res[res].index.values
                if len(id_zone > 0):
                    data_type = layer
                    accuracy = row.h_accuracy
                    latitude = row.latitude
                    longitude = row.longitude
                    id_zone = id_zone[0]
                    timestamp = row.time[:23]
                    print(id_zone)
                    
                    data_types.append(data_type)
                    id_zones.append(id_zone)
                    h_accuracies.append(accuracy)
                    longitudes.append(longitude)
                    latitudes.append(latitude)
                    timestamps.append(timestamp)

    except:
        print("!!! An exception occurred")
    print("\n")

C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:

arbres
0


avaloirs
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


bancs


barri_res_points
0
0
0
0
0
0
0


blocs_de_pierres_ou_b_ton
0
0
0


borne_amovible
0


bornes_incendie
0


camera
0


fontaine


jardini_res


Panneaux


passage_pi_tons


Potelet
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


trottoir_ligne


trottoir_poly
!!! An exception occurred


_clairage


Photos


Tracks


TrackPoints


android_metadata




C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Sextius 1 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:

In [31]:
d = {'data_type':data_types, 'id_zone' : id_zones, 'h_accuracy':h_accuracies, 'longitude':longitudes, 'latitude':latitudes, 'timestamp':timestamps}
df1 = pd.DataFrame(data = d)
df1.head()

,data_type,id_zone,h_accuracy,longitude,latitude,timestamp
0,arbres,0,0.12,5.443566,43.531372,29-07-2024 10:57:43.600
1,avaloirs,0,0.01,5.443638,43.531397,29-07-2024 09:44:54.800
2,avaloirs,0,0.01,5.443757,43.531496,29-07-2024 09:47:58.800
3,avaloirs,0,0.01,5.443655,43.531383,29-07-2024 09:56:05.200
4,avaloirs,0,0.01,5.443604,43.531417,29-07-2024 09:56:25.200


## data 2

In [32]:
data_types = list()
id_zones = list()
h_accuracies = list()
longitudes = list()
latitudes = list()
timestamps = list()

layers = fiona.listlayers(gpkg2)

for layer in layers:
    print(layer)
    try:
        data = gpd.read_file(gpkg2, layer = layer)
        cols = data.columns
        if 'h_accuracy' in cols:
            n = data.shape[0]
            for i in range(n):
                row = data.iloc[i]
                res = row.geometry.intersects(zones.geometry)
                id_zone = res[res].index.values
                if len(id_zone > 0):
                    data_type = layer
                    accuracy = row.h_accuracy
                    latitude = row.latitude
                    longitude = row.longitude
                    id_zone = id_zone[0]
                    timestamp = row.time[:23]
                    print(id_zone)
                    
                    data_types.append(data_type)
                    id_zones.append(id_zone)
                    h_accuracies.append(accuracy)
                    longitudes.append(longitude)
                    latitudes.append(latitude)
                    timestamps.append(timestamp)

    except:
        print("!!! An exception occurred")
    print("\n")
    
d = {'data_type':data_types, 'id_zone' : id_zones, 'h_accuracy':h_accuracies, 'longitude':longitudes, 'latitude':latitudes, 'timestamp':timestamps}
df2 = pd.DataFrame(data = d)
df2.head()

arbre
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


arceaux_v_lo


bancs
2
2
2
2
2


barri_re
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


borne_incendie
2


entre_parking
!!! An exception occurred


fontaine
!!! An exception occurred


jardin_publique


lampadaire
2
2
2
2
2


place_parking
!!! An exception occurred


poubelle
2
2
3
2
3
3


trottoir_ligne


Photos


android_metadata




C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib\site-packages\pyogrio\raw.py:196: RuntimeWarning: GPKG: bad application_id=0x00000000 on '../terrain_data/raw/Aix Faubourg/Cours Sextius 2 GPKG.gpkg'
  return ogr_read(
C:\Git-Repos\CIM\venv\Lib

,data_type,id_zone,h_accuracy,longitude,latitude,timestamp
0,arbre,1,0.012,5.443824,43.529645,29-07-2024 14:06:18.574
1,arbre,1,0.012,5.443991,43.529595,29-07-2024 14:12:32.176
2,arbre,1,0.070,5.443877,43.529635,29-07-2024 14:20:56.149
3,arbre,1,0.012,5.443893,43.529589,29-07-2024 14:23:29.777
4,arbre,1,0.026,5.443778,43.529608,29-07-2024 14:32:49.737


## data1 et data2

In [33]:
df = pd.concat([df1, df2])
df

,data_type,id_zone,h_accuracy,longitude,latitude,timestamp
0,arbres,0,0.120,5.443566,43.531372,29-07-2024 10:57:43.600
1,avaloirs,0,0.010,5.443638,43.531397,29-07-2024 09:44:54.800
2,avaloirs,0,0.010,5.443757,43.531496,29-07-2024 09:47:58.800
3,avaloirs,0,0.010,5.443655,43.531383,29-07-2024 09:56:05.200
4,avaloirs,0,0.010,5.443604,43.531417,29-07-2024 09:56:25.200
...,...,...,...,...,...,...
63,poubelle,2,0.072,5.440389,43.530753,29-07-2024 15:54:33.829
64,poubelle,3,0.012,5.441381,43.530045,29-07-2024 16:16:15.356
65,poubelle,2,0.115,5.441315,43.530101,29-07-2024 16:18:55.365
66,poubelle,3,0.012,5.441963,43.529948,29-07-2024 16:27:32.162


In [34]:
df.shape

(121, 6)

In [35]:
df.id_zone.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3])

In [55]:
df.head()

,data_type,id_zone,h_accuracy,longitude,latitude,timestamp
0,arbres,0,0.12,5.443566,43.531372,29-07-2024 10:57:43.600
1,avaloirs,0,0.01,5.443638,43.531397,29-07-2024 09:44:54.800
2,avaloirs,0,0.01,5.443757,43.531496,29-07-2024 09:47:58.800
3,avaloirs,0,0.01,5.443655,43.531383,29-07-2024 09:56:05.200
4,avaloirs,0,0.01,5.443604,43.531417,29-07-2024 09:56:25.200


## Stats via DuckDB
- Calculer la durée par zone
- Calculer l'erreur moyenne par zone
- Faire des graphiques par zone
- Faire des précisions par type d'objet

In [36]:
df2 = df

In [37]:
import duckdb

In [38]:
duckdb.sql("SELECT * FROM df2")

┌────────────┬─────────┬────────────┬─────────────┬──────────────┬─────────────────────────┐
│ data_type  │ id_zone │ h_accuracy │  longitude  │   latitude   │        timestamp        │
│  varchar   │  int64  │   double   │   double    │    double    │         varchar         │
├────────────┼─────────┼────────────┼─────────────┼──────────────┼─────────────────────────┤
│ arbres     │       0 │       0.12 │ 5.443566463 │  43.53137224 │ 29-07-2024 10:57:43.600 │
│ avaloirs   │       0 │       0.01 │ 5.443637532 │ 43.531396688 │ 29-07-2024 09:44:54.800 │
│ avaloirs   │       0 │       0.01 │ 5.443757118 │ 43.531495655 │ 29-07-2024 09:47:58.800 │
│ avaloirs   │       0 │       0.01 │ 5.443654563 │ 43.531382735 │ 29-07-2024 09:56:05.200 │
│ avaloirs   │       0 │       0.01 │ 5.443603877 │ 43.531416915 │ 29-07-2024 09:56:25.200 │
│ avaloirs   │       0 │       0.01 │ 5.443667367 │ 43.531415283 │ 29-07-2024 09:56:41.200 │
│ avaloirs   │       0 │      0.012 │ 5.443687155 │  43.53142821 │ 29-

In [57]:
duckdb.sql("SELECT * FROM df2 where data_type = 'arbres'")

┌───────────┬─────────┬────────────┬─────────────┬─────────────┬─────────────────────────┐
│ data_type │ id_zone │ h_accuracy │  longitude  │  latitude   │        timestamp        │
│  varchar  │  int64  │   double   │   double    │   double    │         varchar         │
├───────────┼─────────┼────────────┼─────────────┼─────────────┼─────────────────────────┤
│ arbres    │       0 │       0.12 │ 5.443566463 │ 43.53137224 │ 29-07-2024 10:57:43.600 │
└───────────┴─────────┴────────────┴─────────────┴─────────────┴─────────────────────────┘

In [56]:
duckdb.sql("SELECT * FROM df2 where data_type = 'arbre'")

┌───────────┬─────────┬────────────┬─────────────┬──────────────┬─────────────────────────┐
│ data_type │ id_zone │ h_accuracy │  longitude  │   latitude   │        timestamp        │
│  varchar  │  int64  │   double   │   double    │    double    │         varchar         │
├───────────┼─────────┼────────────┼─────────────┼──────────────┼─────────────────────────┤
│ arbre     │       1 │      0.012 │ 5.443823712 │ 43.529645352 │ 29-07-2024 14:06:18.574 │
│ arbre     │       1 │      0.012 │ 5.443990912 │  43.52959522 │ 29-07-2024 14:12:32.176 │
│ arbre     │       1 │       0.07 │ 5.443877327 │  43.52963492 │ 29-07-2024 14:20:56.149 │
│ arbre     │       1 │      0.012 │ 5.443893053 │  43.52958946 │ 29-07-2024 14:23:29.777 │
│ arbre     │       1 │      0.026 │ 5.443777742 │ 43.529608458 │ 29-07-2024 14:32:49.737 │
│ arbre     │       1 │      0.044 │ 5.443755972 │ 43.529618958 │ 29-07-2024 14:47:02.519 │
│ arbre     │       1 │      0.012 │ 5.443731878 │ 43.529672625 │ 29-07-2024 14:

In [58]:
duckdb.sql("SELECT id_zone, count(*) nb FROM df2 group by id_zone")

┌─────────┬───────┐
│ id_zone │  nb   │
│  int64  │ int64 │
├─────────┼───────┤
│       0 │    53 │
│       2 │    39 │
│       1 │     7 │
│       3 │    22 │
└─────────┴───────┘

SELECT strptime('Monday, 2 March 1992 - 08:32:45 PM', '%A, %-d %B %Y - %I:%M:%S %p');

In [42]:
duckdb.sql("SELECT strptime('Monday, 2 March 1992 - 08:32:45 PM', '%A, %-d %B %Y - %I:%M:%S %p');")

┌───────────────────────────────────────────────────────────────────────────────┐
│ strptime('Monday, 2 March 1992 - 08:32:45 PM', '%A, %-d %B %Y - %I:%M:%S %p') │
│                                   timestamp                                   │
├───────────────────────────────────────────────────────────────────────────────┤
│ 1992-03-02 20:32:45                                                           │
└───────────────────────────────────────────────────────────────────────────────┘

In [43]:
duckdb.sql("SELECT strptime('29-07-2024 10:07:27.600', '%d-%m-%Y %H:%M:%S.%f');")

┌─────────────────────────────────────────────────────────────┐
│ strptime('29-07-2024 10:07:27.600', '%d-%m-%Y %H:%M:%S.%f') │
│                          timestamp                          │
├─────────────────────────────────────────────────────────────┤
│ 2024-07-29 10:07:27.0006                                    │
└─────────────────────────────────────────────────────────────┘

In [44]:
duckdb.sql("create or replace table df as (select * replace(strptime(timestamp, '%d-%m-%Y %H:%M:%S.%f') as timestamp) from df2);")

In [59]:
duckdb.sql("""
create or replace table stats as (
SELECT id_zone, 
min(timestamp::TIMESTAMP) from_time,
max(timestamp::TIMESTAMP) to_time, 
to_time-from_time as duration, 
count(*) nb, 
mean(h_accuracy ) as mean_accuracy,
median(h_accuracy) as median_accuracy,
count(*) FILTER (h_accuracy <= 0.01) as accuracy_lower_than_1cm,
count(*) FILTER (h_accuracy <= 0.012) as accuracy_lower_than_1_2cm,
count(*) FILTER (h_accuracy <= 0.015) as accuracy_lower_than_1_25cm,
count(*) FILTER (h_accuracy <= 0.015) as accuracy_lower_than_1_5cm
from df
group by id_zone
order by nb desc)
""")

In [60]:
duckdb.sql("select * from stats")

┌─────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ id_zone │         maxt         │ … │ accuracy_lower_tha…  │ accuracy_lower_tha…  │ accuracy_lower_tha…  │
│  int64  │      timestamp       │   │        int64         │        int64         │        int64         │
├─────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│       0 │ 2024-07-29 10:57:4…  │ … │                   47 │                   50 │                   50 │
│       2 │ 2024-07-29 16:19:5…  │ … │                   21 │                   24 │                   24 │
│       3 │ 2024-07-29 16:27:3…  │ … │                    6 │                    7 │                    7 │
│       1 │ 2024-07-29 14:54:5…  │ … │                    4 │                    4 │                    4 │
├─────────┴──────────────────────┴───┴──────────────────────┴──────────────────────┴──────────────────────┤
│ 4 rows                    

In [61]:
duckdb.sql("select id_zone, accuracy_lower_than_1cm, accuracy_lower_than_1_2cm, accuracy_lower_than_1_25cm, accuracy_lower_than_1_5cm from stats")

┌─────────┬───────────────────────┬───────────────────────────┬────────────────────────────┬───────────────────────────┐
│ id_zone │ accuracy_lower_than…  │ accuracy_lower_than_1_2cm │ accuracy_lower_than_1_25cm │ accuracy_lower_than_1_5cm │
│  int64  │         int64         │           int64           │           int64            │           int64           │
├─────────┼───────────────────────┼───────────────────────────┼────────────────────────────┼───────────────────────────┤
│       0 │                    44 │                        47 │                         50 │                        50 │
│       2 │                     0 │                        21 │                         24 │                        24 │
│       3 │                     0 │                         6 │                          7 │                         7 │
│       1 │                     0 │                         4 │                          4 │                         4 │
└─────────┴─────────────────────

In [62]:
duckdb.sql("select id_zone, nb, duration, epoch(duration)/nb/60 as minutes_by_object from stats;")

┌─────────┬───────┬─────────────────┬────────────────────┐
│ id_zone │  nb   │    duration     │ minutes_by_object  │
│  int64  │ int64 │    interval     │       double       │
├─────────┼───────┼─────────────────┼────────────────────┤
│       0 │    53 │ 01:12:48.9998   │ 1.3738993081761004 │
│       2 │    39 │ 01:02:09.999574 │  1.594016911965812 │
│       3 │    22 │ 00:11:22.00037  │  0.516666946969697 │
│       1 │     7 │ 00:48:36.99976  │  6.945237523809524 │
└─────────┴───────┴─────────────────┴────────────────────┘

On voit qu'on a passé presque 7 minutes dans la seconde zone.

On passe presque 1 minute par objet dans des conditions de RTK fixes.

Attention, il faut prendre en compte le fait que des objets comme les trottoirs et autres surfaces n'ont pas pu être lues par le script. Le nombre d'objet est donc plus grand qu'indiqué.

Dans la première zone, la dernière digitalisation concerne les trottoirs qui ne sont pas présent dans le script.

In [47]:
duckdb.sql("select id_zone, COLUMNS('.*accuracy.*') from stats;")

┌─────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ id_zone │    mean_accuracy     │ … │ accuracy_lower_tha…  │ accuracy_lower_tha…  │ accuracy_lower_tha…  │
│  int64  │        double        │   │        int64         │        int64         │        int64         │
├─────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│       0 │  0.01452830188679246 │ … │                   47 │                   50 │                   50 │
│       2 │  0.04628205128205128 │ … │                   21 │                   24 │                   24 │
│       3 │  0.07336363636363637 │ … │                    6 │                    7 │                    7 │
│       1 │ 0.026857142857142857 │ … │                    4 │                    4 │                    4 │
├─────────┴──────────────────────┴───┴──────────────────────┴──────────────────────┴──────────────────────┤
│ 4 rows                    

In [48]:
duckdb.sql("select id_zone, nb, mean_accuracy, median_accuracy from stats;")

┌─────────┬───────┬──────────────────────┬─────────────────┐
│ id_zone │  nb   │    mean_accuracy     │ median_accuracy │
│  int64  │ int64 │        double        │     double      │
├─────────┼───────┼──────────────────────┼─────────────────┤
│       0 │    53 │  0.01452830188679246 │            0.01 │
│       2 │    39 │  0.04628205128205128 │           0.012 │
│       3 │    22 │  0.07336363636363637 │           0.095 │
│       1 │     7 │ 0.026857142857142857 │           0.012 │
└─────────┴───────┴──────────────────────┴─────────────────┘

Dans la première zone (0), on relève les meilleures précisions.

In [49]:
duckdb.sql("select id_zone, nb, COLUMNS('accuracy_lower.*') from stats;")

┌─────────┬───────┬──────────────────────┬──────────────────────┬──────────────────────────┬───────────────────────────┐
│ id_zone │  nb   │ accuracy_lower_tha…  │ accuracy_lower_tha…  │ accuracy_lower_than_1_…  │ accuracy_lower_than_1_5cm │
│  int64  │ int64 │        int64         │        int64         │          int64           │           int64           │
├─────────┼───────┼──────────────────────┼──────────────────────┼──────────────────────────┼───────────────────────────┤
│       0 │    53 │                   44 │                   47 │                       50 │                        50 │
│       2 │    39 │                    0 │                   21 │                       24 │                        24 │
│       3 │    22 │                    0 │                    6 │                        7 │                         7 │
│       1 │     7 │                    0 │                    4 │                        4 │                         4 │
└─────────┴───────┴─────────────

On relève d'ailleurs  un pourcentage assez élevé de mesures plus basses que 1cm.

In [50]:
duckdb.sql("select count(*) from df where h_accuracy <= 0.01;")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           44 │
└──────────────┘

Elles sont toutes dans la première zone.

In [51]:
duckdb.sql("select min(h_accuracy) min_accuracy_zone2 from df where id_zone = 2;")

┌────────────────────┐
│ min_accuracy_zone2 │
│       double       │
├────────────────────┤
│              0.012 │
└────────────────────┘

La précision minimale sur la dernière zone est de 1.2 cm

In [52]:
duckdb.sql("select count(*) as nb_zone2_inf_1_2cm from df where h_accuracy = 0.012 and id_zone = 2;")

┌────────────────────┐
│ nb_zone2_inf_1_2cm │
│       int64        │
├────────────────────┤
│                 21 │
└────────────────────┘

In [53]:
duckdb.sql("select count(*) as nb_zone2 from df where id_zone = 2;")

┌──────────┐
│ nb_zone2 │
│  int64   │
├──────────┤
│       39 │
└──────────┘

On a 21 objets sur 39 avec une précision de 1.2 cm dans la troisièmle zone.

In [63]:
duckdb.sql("""
select 
data_type, 
mean(h_accuracy) as mean_accuracy, count(*) as nb,
count(*) FILTER (id_zone = 0) as zone0,
count(*) FILTER (id_zone = 1) as zone1,
count(*) FILTER (id_zone = 2) as zone2,
count(*) FILTER (id_zone = 3) as zone3
from df 
group by data_type 
order by mean_accuracy asc;
""")

┌───────────────────────────┬──────────────────────┬───────┬───────┬───────┬───────┬───────┐
│         data_type         │    mean_accuracy     │  nb   │ zone0 │ zone1 │ zone2 │ zone3 │
│          varchar          │        double        │ int64 │ int64 │ int64 │ int64 │ int64 │
├───────────────────────────┼──────────────────────┼───────┼───────┼───────┼───────┼───────┤
│ borne_amovible            │                 0.01 │     1 │     1 │     0 │     0 │     0 │
│ bornes_incendie           │                 0.01 │     1 │     1 │     0 │     0 │     0 │
│ blocs_de_pierres_ou_b_ton │                 0.01 │     3 │     3 │     0 │     0 │     0 │
│ Potelet                   │ 0.010304347826086958 │    23 │    23 │     0 │     0 │     0 │
│ barri_res_points          │ 0.010428571428571428 │     7 │     7 │     0 │     0 │     0 │
│ borne_incendie            │                0.012 │     1 │     0 │     0 │     1 │     0 │
│ bancs                     │               0.0124 │     5 │     0 │  

Les lampadaires, arbres, barrières et poubelles ont la plus basse précision.

Notamment car dans la dernière zone, la précision était assez médiocre à cause de l'effet de canyon urbain.

Il est étrange que les potelets aient une bonne précision mais ces derniers sont recensés essentiellement dans la première zone.

In [65]:
pts = gpd.points_from_xy(df.longitude, df.latitude)

In [67]:
gdf = gpd.GeoDataFrame(
    df, geometry=pts, crs="EPSG:4326"
)

In [68]:
gdf.to_file("Sextius-Points.gpkg")